In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Задача
1. Нагенерировать признаки 
1. Cделать их визуализацию
1. Построить модель для определения человека

> Библиотеки

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

> Работа с данными

In [ ]:
df = pd.read_csv("../input/catch-me-if-you-can/train_sessions.csv", index_col='session_id')
df = df.sort_values(by='time1')
df.head()

In [ ]:
X = df.iloc[:,:-1]
y = df.target

> Набор данных содержит:
1. site_i - индексы посещенных сайтов
1. time_i - время посещения сайтов
1. target - целевая переменная

In [ ]:
#функция для вывода информации по данным
def description(df):
    print(f'Dataset Shape:{df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values   
    summary['Uniques'] = df.nunique().values
    return summary
print('Описание данных:')
description(df)

**Выводы**
1. Колонки time_i имею тип object, а не data-time
1. Имеется много пустых значений, их нужно будет заменить

In [ ]:
times = ['time%s' % i for i in range(1,11)]
df[times] = df[times].apply(pd.to_datetime)
df.get_ftype_counts()

In [ ]:
site = ['site%s' % i for i in range(1,11)]
df[site] = df[site].fillna(0).astype(np.int)

> Новые переменные

1. weekday - день недели
1. year - год
1. hour - часы
1. morning - утро с 6 до 12
1. daytime - день с 12 до 6
1. evening - вечер с 6 до 24
1. night - ночь с 24 до 6

In [ ]:
df['weekday'] = df['time1'].dt.dayofweek
df['year'] = df['time1'].dt.year
df['hour'] = df['time1'].dt.hour
df['morning'] = ((df.hour >= 6) & (df.hour < 12)).astype(int)
df['daytime'] = ((df.hour >= 12) & (df.hour < 18)).astype(int)
df['evening'] = ((df.hour >= 18) & (df.hour < 24)).astype(int)
df['night']   = ((df.hour >= 24) & (df.hour < 6)).astype(int)
df.head()

In [ ]:
#Переменный для визуализации
session_hour = df['hour'].values
session_week = df['weekday'].values
session_year = df['year'].values

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1, 2, 1)
sns.countplot(x = session_hour)
plt.title('Все пользователи')

plt.subplot(1, 2, 2)
sns.countplot(session_hour[y==1])
plt.title('Иван')

plt.show()

**Большинство пользователей сидя в интернете утром, а Иван в вечернее время**

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1, 2, 1)
sns.countplot(x = session_week)
plt.title('Все пользователи')

plt.subplot(1, 2, 2)
sns.countplot(session_week[y==1])
plt.title('Иван')

plt.show()

**У пользователей самый пик использования в среду, у Ивана наоборот**

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1, 2, 1)
sns.countplot(x = session_year)
plt.title('Все пользователи')

plt.subplot(1, 2, 2)
sns.countplot(session_year[y==1])
plt.title('Иван')

plt.show()

**Иван пользовался в интеренете в 2013 и в 2014 года, нежели д**

In [ ]:
new_df = df.drop(['time1','time2','time3','time4','time5','time6','time7','time8','time9','time10'],axis=1)
new_df.head()

In [ ]:
X_new = new_df.drop(['target'], axis=1)
y_new = new_df.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.33, random_state=42)

In [ ]:
grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}
logreg=LogisticRegression()
logreg_cv=GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(X_train,y_train)

In [ ]:
print(logreg_cv.best_score_)
print(logreg_cv.best_params_)

In [ ]:
pred = logreg_cv.predict_proba(X_test)[:,1]

In [ ]:
roc_auc_score(y_test, pred)